# Welcome to the FabSim3 automation toolkit tutorial

[FabSim3](https://fabsim3.readthedocs.io/en/latest/) is a Python-based automation toolkit for scientific simulation and data processing workflows, licensed under the BSD 3-clause license. It is a component of the [VECMA Toolkit](http://www.vecma-toolkit.eu), which is developed as a part of the [VECMA](http://www.vecma.eu) project funded by the European Union Horizon 2020 research and innovation programme.

FabSim3 supports the use of simple one-liner commands to:

- Organize input, output and environment information, creating a consistent log and making it possible by default to repeat/reproduce runs.
- Establish and run coupled models using the workflow automation functionalities.
- Perform large ensemble simulations (or replicated ones) using a one-line command.
- Enable the execution of simulation and analysis tasks on supercomputers.

Users can perform complex remote tasks from a local command-line, and run single jobs, ensembles of multiple jobs, and dynamic workflows through schedulers such as SLURM, Portable Batch System Professional (PBSPro), LoadLeveller and Quality in Cloud and Grid [(QCG)](http://www.qoscosgrid.org/trac/qcg). FabSim3 stores machine-specific configurations in the repository, and applies it to all applications run on that machine. These configurations are updated by any contributor who feels that a fix or improvement is required.

- #### <p> <span style="color:red"><em> <b> NOTE </b> This setup targets VECMA PSNC Jupyter Notebook platform, if you want to use this tutorial on your local PC, please use FabSim3_FASC_PC.ipynb file</em> text</span>.</p>

## <span style="color:#800000"><b> Intalling FabSim3 <b></span>

To install [FabSim3](https://github.com/djgroen/FabSim3), clone the repository using:

In [ ]:
%%bash

if [ ! -d "FabSim3" ]
then
    git clone https://github.com/djgroen/FabSim3
    echo "FabSim3 rep cloned in -> " $PWD
else
    git -C FabSim3 pull
    echo "updating FabSim3 rep cloned in -> " $PWD
fi

### <span style="color:#800000"><b> Setup ssh key <b></span>


setup ssh connection for executing jobs on localhost   

In [ ]:
%%bash

PORT=2222

/usr/sbin/sshd -f /etc/ssh-jovyan/sshd_config
rm -rf ~/.ssh
mkdir ~/.ssh
ssh-keygen -t rsa -N "" -f ~/.ssh/id_rsa
cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys
chmod og-wx ~/.ssh/authorized_keys
ssh-keyscan -4 -p $PORT localhost >> ~/.ssh/known_hosts


### <span style="color:#800000"><b> Dependencies <b></span>

FabSim3 relies strongly on the following Python modules:

- [Fabric](http://www.fabfile.org)
- [PyYAML](https://pypi.org/project/PyYAML/) (any version), 
- [ruamel.yaml](https://pypi.org/project/ruamel.yaml/)
- [numpy](https://numpy.org/install/) and
- [cryptography](https://pypi.org/project/cryptography/).

These dependencies are marked down in the `requirements.txt` file, as

`fabric3==1.13.1.post1, 
 pyyaml, 
 pytest, 
 pytest-pep8, 
 ruamel.yaml, 
 numpy, 
 cryptography`
 
To install requitements.txt, simply run the following:

In [ ]:
# Install required Python modules
!pip install -r FabSim3/requirements.txt

### <span style="color:#800000"><b> Configure `machine_user.yml` file <b></span>

To set personal configurations, copy `machines_user_example.yml` as `machines_user.yml` in the `FabSim3/deploy` folder and run the following sequence of commands, which will change `localhost:` configurations:


In [ ]:
%%bash

PORT=2222

# Configure machines_user.yml
cp FabSim3/deploy/machines_user_example.yml FabSim3/deploy/machines_user.yml

# Defines your username from local configuration
sed -i "s/your-username/`whoami`/g" FabSim3/deploy/machines_user.yml

# Defines port number for localhost connection
sed -i "s#localhost:#localhost:\n  port: $PORT#g" FabSim3/deploy/machines_user.yml

# Defines working directory for configs and results directories
sed -i "s#localhost:#localhost:\n  local_configs: \"$PWD/FabSim3/config_files\"#g" FabSim3/deploy/machines_user.yml
sed -i "s#localhost:#localhost:\n  local_results: \"$PWD/FabSim3/results\"#g" FabSim3/deploy/machines_user.yml

# Defines home path for execution
sed -i "s#localhost:#localhost:\n  home_path_template: \"$PWD/FabSim3/localhost_exe\"#g" FabSim3/deploy/machines_user.yml

To view the changes made in the `machines_user.yml` file, simply run:

In [ ]:
import yaml

machines_user_yml = yaml.load(open("FabSim3/deploy/machines_user.yml"), Loader=yaml.FullLoader)
print(yaml.dump({'localhost': machines_user_yml['localhost']}))

### <span style="color:#800000"><b> Add FabSim3 to your PATH and PYTHONPATH <b></span>

Add FabSim3 executable file to system `PATH` and setup `PYTHONPATH`

In [ ]:
import os
import sys

sys.path.insert(0,  r'%s/FabSim3/bin' %(os.getcwd()))
sys.path.insert(0,  r'%s/FabSim3' %(os.getcwd()))
os.environ['PATH']=r'%s/FabSim3/bin' %(os.getcwd())+os.pathsep+os.environ['PATH']

# to test if fabsim command is availble
!which fabsim

## <span style="color:#800000"><b> FabSim3 plugins <b></span>

FabSim3 contains an integrated test infrastructure, more flexible customisation options using a plugin system and  examples to improve usability.

In [ ]:
%%bash

cat FabSim3/deploy/plugins.yml

These plugins will be installed in the `FabSim3/plugins` directory.

### <span style="color:#800000"><b> Dummy instance - FabDummy plugin <b></span>

To demonstrate the use of FabSim3 plugin, install the [FabDummy](https://github.com/djgroen/FabDummy) plugin  by running `install_plugin` command:

In [ ]:
# Install FabDummy
!fabsim localhost install_plugin:FabDummy

#### <span style="color:#800000"><b> Running FabDummy test <b></span>

To test the FabDummy plugin, run a dummy job using:

In [ ]:
# Execute a dummy test job
!fabsim localhost dummy:dummy_test

## <span style="color:#800000"><b> Flu And Coronavirus Simulator (FACS) - FabCovid19 plugin <b></span>
[FACS](https://github.com/djgroen/facs) is an agent-based modelling code that models the spread of flu and coronaviruses in local regions. Up to now, we have used it to model the spread of Covid-19 in a range of London boroughs. For detailed information, see [FACS](https://facs.readthedocs.io/en/latest/) documentation.

To install FACS, clone the repository by running:

In [ ]:
%%bash

# Clone the FACS repository
if [ ! -d "facs" ]
then
    git clone -b master https://github.com/djgroen/facs.git
    echo "facs rep cloned in -> " $PWD
else
    git -C facs pull
    echo "updating facs rep cloned in -> " $PWD
fi

### <span style="color:#800000"><b> Install required python modules for FACS <b></span>

In [ ]:
!pip install pandas matplotlib numpy chaospy easyvvuq

### <span style="color:#800000"><b> Installing FabCovid19 <b></span>

A FabSim3-based FabCovid19 plugin provides an environment for researchers and organisations to construct and modify simulations, instantiate and execute multiple runs for different policy decisions, as well as to validate and visualise the obtained results against the existing data.

To install FabCovid19, run `install_plugin` function using:

In [ ]:
# Install FabCovid19 
!fabsim localhost install_plugin:FabCovid19

#### <span style="color:#800000"><b> Setup FabCovid19 machine specifications <b></span>

To run FACS code by FabCovid19 plugin, we need to add FACS PATH to the `machines_FabCovid19_user.yml` configuration file

In [ ]:
!sed -i "s#facs_location: \"<..>\"#facs_location : \"$PWD/facs\"#g" FabSim3/plugins/FabCovid19/machines_FabCovid19_user.yml

To view changes in `machines_FabCovid19_user`, simply run the following

In [ ]:
machines_user_yml = yaml.load(open("FabSim3/plugins/FabCovid19/machines_FabCovid19_user.yml"), Loader=yaml.FullLoader)

print(yaml.dump({'localhost': machines_user_yml['localhost']}))

#### <span style="color:#800000"><b> Executing a single FACS scenario <b></span>


To execute FabCovid19 job on a localhost, execute one of the boroughs using `FACS` code in `FabSim3/plugins/FabCovid19/config_files`, such as `brent`, `camden`, `ealing`, `hammersmith_fulham`, `harrow`, `hillingdon`, `kensington_chelsea`, `westminster`. 

Since that the total execution of each borough takes more that 50 minutes to be finished, for illustration purposes,  we use a `test` borough to have more faster simulation run which can be done less that 2 minutes.

In [ ]:
!fabsim localhost covid19:test,TS='extend-lockdown',TM=1,quicktest=true

#### <span style="color:#800000"><b> Fetching the simulation results to `FabSim3/results` directory <b></span>

After the job has finished, a message will be printed indicating where the output data resides. To fetch and copy the obtained results to `FabSim3/results` directory, simply execute:

In [ ]:
!fabsim localhost fetch_results

### <span style="color:#800000"><b> SA analysis of FACS <b></span>

To demonstrate the added value offered by VECMAtk more concretely, we showcase one specific VVUQ procedure example, using the Flu And Coronavirus Simulator. We perform sensitivity analysis across six different input parameters of [FACS](https://github.com/djgroen/facs) to identify their sensitivity relative to our quantity of interest (QoI).
In below table, we provide the default value for each parameter along with the range of likely values.

| Parameters                | Type  | Default value | Uniform range  |
|---------------------------|-------|---------------|----------------|
| infection rate            | float | 0.07          | (0.0035, 0.14) |
| mortality period          | float | 8.0           | (4.0, 16.0)    |
| recovery period           | float | 8.0           | (4.0, 16.0)    |
| mild recovery period      | float | 8.05          | (4.5, 12.5)    |
| incubation period         | float | 3.0           | (2.0, 6.0)     |
| period to hospitalisation | float | 12.0          | (8.0, 16.0     |

We use the [Chaospy](https://pypi.org/project/chaospy) library in EasyVVUQ to generate samples from the input parameters. Specifically, in this example we used the stochastic collocation method with a sparse-grid sampling plan of 13 samples, which we then convert to simulation inputs using the EasyVVUQ encoder, and execute them using FabSim3.
```python
import chaospy as cp
...
...
...
# parameters to vary
vary = {
    "infection_rate": cp.Uniform(0.0035, 0.14),
    "mortality_period": cp.Uniform(4.0, 16.0),
    "recovery_period": cp.Uniform(4.0, 16.0),
    "mild_recovery_period": cp.Uniform(4.5, 12.5),
    "incubation_period": cp.Uniform(2.0, 6.0),
    "period_to_hospitalisation": cp.Uniform(8.0, 16.0),
}

# create SCSampler (stochastic collocation)
sampler = uq.sampling.SCSampler(vary=vary,
                                polynomial_order=2,
                                quadrature_rule="C",
                                sparse=True,
                                growth=True,
                                midpoint_level1=True
                                )
...
...
...
```    




#### <span style="color:#800000"><b> Run SA FabCovid19  <b></span>

To execute SA analysis of FACS using FabCovid19 plugin, simply run the following.


In [ ]:
!fabsim localhost covid19_init_SC:test,TS='extend-lockdown',TM=1,quicktest=True

#### <span style="color:#800000"><b> Analysis output results   <b></span>

Once execution has concluded, we then decode and collate the results and perform a Sobol sensitivity analysis relative to our QoI (number of deaths over time).

##### <span style="color:#800000"><b> Fetching results   <b></span>

In [ ]:
# Fetching results
!fabsim localhost fetch_results

##### <span style="color:#800000"><b> Run SA Analysis   <b></span>
    
The output generated files and figures will be placed in `FabSim3/plugins/FabCovid19/covid19_test_easyvvuq_SCSampler` folder.

In [ ]:
!fabsim localhost covid19_analyse_SC:test

To view generated plots, simply run:

In [ ]:
from IPython.display import Image 
Image('FabSim3/plugins/FabCovid19/covid19_test_easyvvuq_SCSampler/plot_first_order_sobol_index[dead].png',width = 800)